### Sequence Models and LSTM

In [11]:
import torch
torch.manual_seed(1)

In [2]:
lstm = torch.nn.LSTM(3, 3)  # Input은 3차원, output도 3차원
inputs = [torch.autograd.Variable(torch.randn(1, 3))
          for _ in range(5)]  # sequence의 길이는 5

# Hidden state를 초기화한다.
hidden = (torch.autograd.Variable(torch.randn(1, 1, 3)),
          torch.autograd.Variable(torch.randn((1, 1, 3))))
for i in inputs:
    # sequence의 요소 하나씩 단계적으로 진행한다.
    # 매 단계마다 hidden은 hidden state 정보를 담게 된다.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

In [3]:
# Sequence 전체를 한 번에 진행해도 똑같은 결과를 얻을 수 있다.
# LSTM이 주는 output 중 첫 번째 값인 "out"은 sequence 모두를 진행하면서 얻게 되는
# 모든 hidden state를 담고 있다.
# 두 번째 output인 "hidden"은 가장 최근의 hidden state만을 갖고 있다.
# ("out"의 마지막과 "hidden"의 값이 같은 것을 확인해보자)
# "out"을 통해 sequence 전체의 hidden state에 접근할 수 있는 것에 가치가 있고,
# "hidden"은 LSTM의 argument로 쓰여서 sequence의 backpropagate를 계속 진행할 수 있게 해주는 것이다.

# LSTM에 넣기 위해 두 번째 축을 추가해서 3D tensor로 만들자.
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
# Hidden state를 초기화한다.
hidden = (torch.autograd.Variable(torch.randn(1, 1, 3)),
          torch.autograd.Variable(torch.randn((1, 1, 3))))
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]])
(tensor([[[-0.3368,  0.0959, -0.0538]]]), tensor([[[-0.9825,  0.4715, -0.0633]]]))


#### LSTM for part-of-speech tagging

- Input 문장을 w1,⋯,wM 이라고 한다.
1) wi∈V, 즉 각 단어 wi는 단어장 V 안에 속해있다.
2) T를 품사 모음으로 표시한다. 그리고 yi를 단어 wi의 품사로 표시한다.
3) 이제 단어 wi의 품사에 대한 우리의 예측을 y^i로 표시한다.

- 이제 예측을 하기 위해서 LSTM 모델에 문장을 던져줘야 한다. 
- i 번째 순서의 hidden state를 hi라고 표시하겠다. 
- 그리고 모든 품사 태그에 고유한 숫자를 부여하겠다. 
- word embedding 예제에서 word_to_ix를 만들었던 것과 똑같은 이치이다. 
- y^i를 계산하기 위한 규칙은 다음과 같다.

- y^i=argmaxj(logSoftmax(Ahi+b))j

In [5]:
import pandas as pd
from collections import Counter
data = pd.read_csv('./nsmc/example.csv', encoding = 'cp949')
ans, pos = data['answer'], data['pos']

In [3]:
ans_text = ans[0].split('.')
pos_text = pos[0].split('.')

In [6]:
#Noun, Verb, Adjective, Else
korean = list()
entity = list()
for elem in pos_text:
    elem_split = elem.split(' ')
    sub_entity = list()
    sub_korean = list()
    for sub_elem in elem_split:
        sub_elem_split = sub_elem.split('/')
        if len(sub_elem_split) > 1:
            sub_korean.append(sub_elem_split[0])
            if sub_elem_split[1] in ['Noun','Verb','Adjective']:
                sub_entity.append(sub_elem_split[1])
            else:
                sub_entity.append('Else')
    korean.append(sub_korean)
    entity.append(sub_entity)

In [7]:
korean_tag = list(Counter(sum(entity,[])).keys())

In [8]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return torch.autograd.Variable(tensor)

In [9]:
training_data = [
    (korean[0], entity[0]),
    (korean[1], entity[1]),
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

tag_to_ix = {"Else": 0, "Noun": 1, "Verb": 2, "Adjective" : 3}

# 아래 값들은 보통 32 또는 64 차원으로 사용하지만
# 여기서는 우리가 train하는 weight들이 어떻게 변하는지 직접 볼 수 있도록
# 작게 설정한다.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'리더': 0, '의': 1, '책임감': 2, '과': 3, '학과': 4, '1': 5, '년': 6, '을': 7, '잘': 8, '경영': 9, '해야': 10, '겠': 11, '다는': 12, '열정': 13, '으로': 14, '문제점': 15, '파악하고': 16, '방안': 17, '마련한': 18, '경험': 19, '이': 20, '있': 21, '습니다': 22, '': 23, '때': 24, '대다수': 25, '구성원': 26, '들': 27, '설득': 28, '했': 29, '던': 30, '과정': 31, '기억': 32, '에': 33, '많이': 34, '남': 35}


In [12]:
class LSTMTagger(torch.nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = torch.nn.Embedding(vocab_size,
                                                  embedding_dim)
        
        # LSTM은 word embedding과 hidden 차원값을 input으로 받고,
        # hidden state를 output으로 내보낸다.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)
        
        # Hidden state space에서 tag space로 보내는 linear layer를 준비한다.
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # Hidden state는 자동적으로 만들어지지 않으므로 직접 기능을 만들겠다.
        # 3D tensor의 차원은 각각 (layer 개수, mini-batch 개수, hidden 차원)
        # 을 의미한다. 왜 이렇게 해야만 하는지 궁금하다면 Pytorch 문서를 참고 바란다.
        return (
            torch.autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
            torch.autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
        )
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1),
            self.hidden
        )
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = torch.nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores

In [13]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM,
                   len(word_to_ix), len(tag_to_ix))
loss_function = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Training하기 전의 품사 태깅 점수를 보겠다.
# Output의 (i, j) 원소는 i번째 단어가 j번째 품사일 점수를 나타낸다.
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
print(tag_scores)

tensor([[-1.4375, -1.4504, -1.6374, -1.0980],
        [-1.4177, -1.4590, -1.5461, -1.1641],
        [-1.3452, -1.3871, -1.6143, -1.2356],
        [-1.3194, -1.3969, -1.6079, -1.2551],
        [-1.2656, -1.4696, -1.6918, -1.1917],
        [-1.4048, -1.4731, -1.5217, -1.1808],
        [-1.3910, -1.3842, -1.5444, -1.2476],
        [-1.3512, -1.3893, -1.5865, -1.2476],
        [-1.3708, -1.3502, -1.5670, -1.2793],
        [-1.3624, -1.3672, -1.5820, -1.2602],
        [-1.2968, -1.3721, -1.6458, -1.2724],
        [-1.2338, -1.4749, -1.6644, -1.2353],
        [-1.4441, -1.4147, -1.5423, -1.1804],
        [-1.4133, -1.4408, -1.6053, -1.1422],
        [-1.3752, -1.4578, -1.5350, -1.2073],
        [-1.4161, -1.4195, -1.5363, -1.2029],
        [-1.3583, -1.4408, -1.5634, -1.2149],
        [-1.3898, -1.3574, -1.5578, -1.2626],
        [-1.3647, -1.3308, -1.6248, -1.2612],
        [-1.3178, -1.3701, -1.5977, -1.2880],
        [-1.3256, -1.3452, -1.6077, -1.2964],
        [-1.4257, -1.3959, -1.5892

In [14]:
for epoch in range(300):
    for sentence, tags in training_data:
        # Step 1. Torch에서 gradient는 축적된다는 기억하자.
        # 새로운 데이터를 넣기 전에, 기존 gradient 정보를 날려줘야 한다.
        model.zero_grad()
        
        # 또한 LSTM의 이전 단계 hidden state와 분리시키면서
        # hidden state를 초기화해줘야 한다.
        model.hidden = model.init_hidden()
        
        # Step 2. Network에 넣을 수 있도록 input 자료를 알맞게 변환해준다.
        # 즉, 단어 인덱스에 맞게 Variable로 변환해준다.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        # Step 3. Forward pass를 돌려라.
        tag_scores = model(sentence_in)
        
        # Step 4. Loss, gradient를 계산하고,
        # optimizer.step()을 통해 parameter를 업데이트한다.
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# Training이 끝난 후의 결과를 보겠다.
inputs = prepare_sequence(training_data[0][0], word_to_ix)
print(training_data[0][0])
tag_scores = model(inputs)
# 출력물의 (i,j)번째 원소는 i번째 단어가 j번째 품사일 점수이다.
# 그 중 최고 점수를 기록한 품사를 선택하게 된다.
print(tag_scores)

['리더', '의', '책임감', '과', '학과', '의', '1', '년', '을', '잘', '경영', '해야', '겠', '다는', '열정', '으로', '문제점', '을', '파악하고', '방안', '을', '마련한', '경험', '이', '있', '습니다']
tensor([[-0.2837, -2.3610, -1.9290, -4.9097],
        [-0.2307, -2.9165, -1.9193, -5.2582],
        [-1.9291, -0.2844, -2.3952, -4.5035],
        [-1.1238, -0.5878, -2.3109, -3.9005],
        [-1.1196, -0.7001, -1.8537, -3.8936],
        [-0.2659, -2.5938, -1.8952, -4.7776],
        [-0.3417, -2.0744, -1.8651, -4.7228],
        [-1.5112, -0.4672, -1.9949, -4.0996],
        [-0.2597, -3.0601, -1.7608, -4.6140],
        [-0.6587, -1.5621, -1.3568, -4.1788],
        [-2.6984, -0.1783, -2.4711, -4.4636],
        [-1.0979, -0.8381, -1.5211, -4.1702],
        [-0.2656, -2.8979, -1.7653, -4.9607],
        [-0.5384, -1.6394, -1.5536, -4.5322],
        [-1.2816, -0.5753, -1.9461, -4.0717],
        [-0.3253, -2.0990, -1.9053, -5.0678],
        [-1.4343, -0.5244, -1.8670, -4.1836],
        [-0.2649, -3.0814, -1.7315, -4.6226],
        [-0.8185, -1.